## Install the packages

In [1]:
!pip install -Uqq datasets pythainlp==2.2.4 transformers==4.4.0 tensorflow==2.4.0 tensorflow_text emoji seqeval sentencepiece fuzzywuzzy
!npx degit --force https://github.com/vistec-AI/thai2transformers#dev

     |████████████████████████████████| 225kB 13.1MB/s 
     |████████████████████████████████| 8.9MB 24.2MB/s 
     |████████████████████████████████| 2.1MB 27.9MB/s 
     |████████████████████████████████| 394.7MB 38kB/s 
     |████████████████████████████████| 4.3MB 47.3MB/s 
     |████████████████████████████████| 133kB 74.9MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 1.2MB 43.4MB/s 
     |████████████████████████████████| 245kB 69.8MB/s 
     |████████████████████████████████| 112kB 71.1MB/s 
     |████████████████████████████████| 747kB 59.7MB/s 
     |████████████████████████████████| 901kB 44.6MB/s 
     |████████████████████████████████| 3.3MB 43.8MB/s 
ERROR: tensorflow-text 2.5.0 has requirement tensorflow<2.6,>=2.5.0, but you'll have tensorflow 2.4.0 which is incompatible.
npx: installed 1 in 2.167s
> destination directory is not empty. Using --force, continuing
> cloned vistec-AI/thai2transformers#dev


In [2]:
%load_ext autoreload
%autoreload 2

import pythainlp, transformers
pythainlp.__version__, transformers.__version__ #fix pythainlp to stabilize word tokenization for metrics

('2.2.4', '4.4.0')

In [3]:
import collections
import logging
import pprint
import re
from tqdm.auto import tqdm

import numpy as np
import torch

#datasets
from datasets import (
    load_dataset, 
    load_metric, 
    concatenate_datasets,
    load_from_disk,
)

#transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)

#thai2transformers
import thai2transformers
from thai2transformers.metrics import (
    squad_newmm_metric,
    question_answering_metrics,
)
from thai2transformers.preprocess import (
    prepare_qa_train_features
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

from tqdm import tqdm

## Load model and predict

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/finetune_thaiSum3')
model_name = "wangchanberta-base-att-spm-uncased" 
tokenizer = AutoTokenizer.from_pretrained(
                f'airesearch/{model_name}',
                revision='main',
                model_max_length=416,)

data_collator = default_data_collator

trainer = Trainer(
  model,
  data_collator=data_collator,
  tokenizer=tokenizer,
)

In [8]:
import pandas as pd

In [6]:
def post_process(data, raw_predictions, tokenizer, n_best_size = 20, max_answer_length=50):
  all_start_logits, all_end_logits = raw_predictions
  predictions = []
  for start_logits, end_logits, example in zip(all_start_logits, all_end_logits, data):
    start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
    end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
    valid_answers = []
    for start_index in start_indexes:
      for end_index in end_indexes:
          # Don't consider answers with a length that is either < 0 or > max_answer_length.
          if end_index < start_index or end_index - start_index + 1 > max_answer_length:
              continue
          valid_answers.append(
              {
                  "score": start_logits[start_index] + end_logits[end_index],
                  "text": tokenizer.decode(example['input_ids'][start_index:end_index], skip_special_tokens=True)
              }
          )
    if len(valid_answers) > 0:
        best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
    else:
        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid failure.
        best_answer = {"text": "", "score": 0.0} 
    predictions.append(best_answer["text"])
  return predictions
  
def predict(texts):
  inp_ids = tokenizer(texts, max_length=416, truncation=True, padding='max_length')['input_ids']
  att_mask = tokenizer(texts, max_length=416, truncation=True, padding='max_length')['attention_mask']
  res = pd.DataFrame(columns=['attention_mask', 'input_ids'])
  res = res.append({'attention_mask': att_mask, 
                    'input_ids': inp_ids}, ignore_index=True)
  res.to_json('text.json', orient='records', lines=True)
  dataset = load_dataset('json', data_files='text.json')
  raw_predictions = trainer.predict(dataset['train'])
  predictions = post_process(dataset['train'], raw_predictions[0], tokenizer)
  if ',' in predictions[0][:10]:
    return ''.join(predictions[0].split(',')[1:])
  return predictions[0]

In [9]:
predict('เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.) ออกมาระบุว่า การเลือกตั้งจะถูกเลื่อนออกไปถึงปี 2561 ว่า ขอให้ไปสอบถามกับ สนช. แต่เชื่อว่าคงไม่กล้าพูดอีก เพราะทำให้คนเข้าใจผิด ซึ่งที่ สนช.พูดเนื่องจากผูกกับกฎหมายของกรรมการร่างรัฐธรรมนูญ(กรธ.) ตนจึงไม่ขอวิพากษ์วิจารณ์ แต่รัฐบาลยืนยันว่ายังเดินตามโรดแม็ป ซึ่งโรดแม็ปมองได้สองแบบ คือ มีลำดับขั้นตอนและการกำหนดช่วงเวลา โดยเริ่มต้นจากการประกาศใช้รัฐธรรมนูญ แต่ขณะนี้รัฐธรรมนูญยังไม่ประกาศใช้ จึงยังเริ่มนับหนึ่งไม่ถูก จากนั้นเข้าสู่ขั้นตอนการร่างกฎหมายประกอบร่างรัฐธรรมนูญหรือกฎหมายลูก ภายใน 240 วัน ก่อนจะส่งกลับให้ สนช.พิจารณา ภายใน 2 เดือน\xa0,นายวิษณุ กล่าวต่อว่า หากมีการแก้ไขก็จะมีการพิจารณาร่วมกับ กรธ.อีก 1 เดือน ก่อนนำขึ้นทูลเกล้าฯ ทรงลงพระปรมาภิไธย ภายใน 90 วัน และจะเข้าสู่การเลือกตั้งภายในระยะเวลา 5 เดือน ซึ่งทั้งหมดนี้คือโรดแม็ปที่ยังเป็นแบบเดิมอยู่ ส่วนเดิมที่กำหนดวันเลือกตั้งไว้ภายในปี 60 นั้น เพราะมาจากสมมติฐานของขั้นตอนเดิมทั้งหมด แต่เมื่อมีเหตุสวรรคตทุกอย่างจึงต้องเลื่อนออกไป ส่วนการพิจารณากฎหมายลูกทั้งหมด 4 ฉบับ ขณะนี้กรธ.พิจารณาแล้วเสร็จ 2 ฉบับ คือ พ.ร.ป.พรรคการเมือง และพ.ร.ป. คณะกรรมการการเลือกตั้ง แต่ พ.ร.ป.การเลือกตั้งควรจะพิจารณาได้เร็วกลับล่าช้า ดังนั้น กรธ.จะต้องออกชี้แจงถึงเหตุผลว่าทำไมพิจารณากฎหมายดังกล่าวล่าช้ากว่ากำหนด ส่งผลให้เกิดข้อสงสัยจนถึงทุกวันนี้ ส่วนกรณีที่ สนช. ระบุว่า มีกฎหมายเข้าสู่การพิจารณาของ สนช.เป็นจำนวนมาก ทำให้ส่งผลกระทบต่อโรดแม็ปนั้น รัฐบาลเคยบอกไว้แล้วว่าในช่วงนี้ของโรดแม็ปกฎหมายจะเยอะกว่าที่ผ่านมา ดังนั้น สนช.จะต้องบริหารจัดการกันเอง เพราะได้มีการเพิ่มสมาชิก สนช.ให้แล้ว.')

Using custom data configuration default-c1b9d09df72fe7ba


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-c1b9d09df72fe7ba/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


'แต่รัฐบาลยืนยันว่ายังเดินตามโรดแม็ป ซึ่ง'